
# Track B — A(LLM) 
**Scope**: A-role only (LLM, Prompts, RAG, Intent).  
**Note**: This notebook uses an in-memory vector store for demo. Replace with pgvector for production.



## 0️⃣ Setup
- Set `OPENAI_API_KEY` (env var or fill in the cell).


In [ ]:
# Minimal imports and client bootstrap (safe, no hard failure)
import os, json, textwrap
from typing import List, Dict, Any
import numpy as np

# Optional imports: guarded to avoid notebook import-time crashes
try:
    from openai import OpenAI
    openai_available = True
except Exception:
    OpenAI = None
    openai_available = False

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if openai_available and OPENAI_API_KEY:
    client = OpenAI(api_key=OPENAI_API_KEY)
else:
    client = None

PDF_PATH = "Track_B_Tenancy_Agreement.pdf"
print("OpenAI available:", openai_available, "| API key set:", bool(OPENAI_API_KEY))
print("PDF path:", PDF_PATH)


## 1️⃣ Prompt Templates
合同问答（带引用、JSON输出）与意图识别（三分类）。


In [ ]:
# 🧩 Fused Prompt Definitions 
SYSTEM_PROMPT = (
    "You are an expert assistant for tenancy contracts. "
    "Use only the information provided in the contract context to answer. "
    "Cite clause numbers (e.g., 'Clause 2(b)') when referencing them. "
    "Never fabricate information that is not explicitly supported by the contract. "
    "If uncertain, say 'Not sure, please check with landlord.' "
    "Always respond concisely in English."
)

CONTRACT_QA_PROMPT = """
You are a tenancy contract assistant. 
Use ONLY the contract excerpts below to answer the user's question.

--- CONTRACT CONTEXT START ---
{context}
--- CONTRACT CONTEXT END ---

Question: {question}

Guidelines:
- Answer based strictly on the given clauses.
- If the topic is not covered, say: "Not sure, please check with landlord."
- Cite clause numbers (e.g., 'Clause 2(b)') where applicable.
- Do NOT guess or fabricate.
- Provide your final answer in JSON format:
{{
  "answer": "...",
  "citations": [{{"clause":"...", "pages":[...]}}],
  "confidence": "high|medium|low"
}}
"""

print("✅ Fused prompt templates loaded.")


## 2️⃣ Intent Classification
温度 0，输出严格为三选一。若未配置 OpenAI，将提示未就绪。


In [ ]:
def classify_intent(user_input: str) -> str:
    if client is None:
        return "(OpenAI not configured)"
    prompt = INTENT_PROMPT.format(user_input=user_input)
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content": prompt}],
        temperature=0
    )
    label = resp.choices[0].message.content.strip().lower()
    if label not in {"contract_qa","repair_request","status_check"}:
        label = "contract_qa"
    return label

print("Intent function ready.")


## 3️⃣ Contract Ingestion (PDF → Chunks → Embeddings)
以 token 数分块并向量化。此处使用内存 `VECTOR_DB` 演示；联调时替换为 **B组** 的数据库接口。


In [ ]:
# Tokenizer (guarded import)
try:
    import tiktoken
    tokenizer = tiktoken.get_encoding("cl100k_base")
    token_ok = True
except Exception:
    tokenizer = None
    token_ok = False

VECTOR_DB: List[Dict[str, Any]] = []

def split_text_by_tokens(text: str, max_tokens=900, overlap=150):
    if not token_ok or tokenizer is None:
        # naive fallback by characters if tiktoken not installed
        text = (text or "").strip()
        if not text:
            return []
        chunks = []
        step = 2500  # rough char window
        o = 400
        for i in range(0, len(text), step - o):
            chunks.append(text[i:i+step])
        return chunks

    text = (text or "").strip()
    if not text:
        return []
    toks = tokenizer.encode(text)
    chunks = []
    start = 0
    while start < len(toks):
        end = min(start + max_tokens, len(toks))
        chunk = tokenizer.decode(toks[start:end])
        chunks.append(chunk)
        if end == len(toks): break
        start = max(0, end - overlap)
    return chunks

def insert_document_chunk(doc_id: str, page: int, content: str, embedding: List[float]):
    VECTOR_DB.append({
        "doc_id": doc_id,
        "page": page,
        "content": content,
        "embedding": np.array(embedding, dtype=np.float32)
    })

def process_contract_pdf(pdf_path: str, doc_id: str = "TA-EXAMPLE"):
    if client is None:
        print("⚠️ OpenAI 未配置，跳过向量化。")
        return
    try:
        import pdfplumber
    except Exception:
        print("⚠️ 未安装 pdfplumber，跳过解析。")
        return
    total = 0
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            for chunk in split_text_by_tokens(text):
                emb = client.embeddings.create(
                    model="text-embedding-3-large",
                    input=chunk
                ).data[0].embedding
                insert_document_chunk(doc_id, i, chunk, emb)
                total += 1
    print(f"✅ 向量化完成：{total} 个片段入库（内存）。")

# Try to ingest if file exists
import os
if os.path.exists(PDF_PATH):
    process_contract_pdf(PDF_PATH)
else:
    print("⚠️ 未找到合同 PDF：", PDF_PATH)


## 4️⃣ RAG Retrieval
Top-K 相似片段 → 拼接上下文 → 生成 JSON 答案（含引用）。


In [ ]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    if denom == 0:
        return 0.0
    return float(np.dot(a, b) / denom)

def get_similar_chunks(query: str, top_k: int = 5) -> List[Dict[str,Any]]:
    if client is None or not VECTOR_DB:
        return []
    q_emb = client.embeddings.create(
        model="text-embedding-3-large",
        input=query
    ).data[0].embedding
    q_emb = np.array(q_emb, dtype=np.float32)
    scored = [(cosine_similarity(q_emb, row["embedding"]), row) for row in VECTOR_DB]
    scored.sort(key=lambda x: x[0], reverse=True)
    return [row for _, row in scored[:top_k]]

def build_context(chunks: List[Dict[str,Any]]) -> str:
    segs = []
    for c in chunks:
        snippet = textwrap.shorten(c["content"].strip().replace("\n"," "), width=600, placeholder="...")
        segs.append(f"[Doc {c['doc_id']} p.{c['page']}] {snippet}")
    return "\n\n".join(segs)

def generate_rag_answer(question: str, top_k: int = 5) -> dict:
    if client is None:
        return {"answer":"(OpenAI not configured)", "citations":[], "confidence":"low"}
    chunks = get_similar_chunks(question, top_k=top_k)
    context = build_context(chunks)
    prompt = CONTRACT_QA_PROMPT.format(context=context, question=question)
    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content": prompt}],
            temperature=0.2
        )
        raw = resp.choices[0].message.content.strip()
        # Try to parse JSON
        try:
            data = json.loads(raw)
        except json.JSONDecodeError:
            start, end = raw.find("{"), raw.rfind("}")
            if start != -1 and end != -1 and end > start:
                try:
                    data = json.loads(raw[start:end+1])
                except Exception:
                    data = {"answer": raw, "citations": [], "confidence": "medium"}
            else:
                data = {"answer": raw, "citations": [], "confidence": "medium"}
        return data
    except Exception as e:
        return {"answer": f"Error calling LLM: {e}", "citations": [], "confidence": "low"}

print("RAG functions ready.")


## 5️⃣ Unified `generate_reply`
按意图分流：合同问答 → RAG；报修/查询 → 返回指引（交给 B/C 表单与API继续处理）。


In [ ]:
def generate_reply(message: str) -> str:
    intent = classify_intent(message)
    if intent == "(OpenAI not configured)":
        return intent
    if intent == "contract_qa":
        data = generate_rag_answer(message, top_k=5)
        return f"""{data.get('answer','')}
\n📄 Source: {data.get('citations', [])}"""
    elif intent == "repair_request":
        return "🛠️ Repair request noted. Please provide location, issue type, urgency, and photo link."
    elif intent == "status_check":
        return "🔎 Please provide your ticket number (e.g., T2025-001)."
    else:
        return "❓ I’m not sure I understood. Could you rephrase?"

print("generate_reply ready.")


## 6️⃣ Quick Tests
若未配置 OpenAI 或未导入 PDF，只会返回占位信息；配置后可得到完整答案与引用。


In [ ]:
tests = [
    "When is my rent due?",
    "What is the amount of security deposit?",
    "Can I terminate the lease early?",
    "The toilet is leaking, what should I do?",
]
for q in tests:
    print("Q:", q)
    print("A:", generate_reply(q))
    print("-"*80)


## 7️⃣ Handover Notes (for B/C)
- B(DB) provide: `insert_document_chunks(...)` and `get_similar_chunks(...)` for pgvector.
- C(API) wrap `generate_reply(message)` under `/ask` and handle auth/roles.



## ✅ Summary
- Prompts, Intent, Ingestion, RAG, Unified reply implemented.
- Replace in-memory store with pgvector & connect API for production.

Generated at: 2025-10-27 12:16:08


**Version:** v3 (Fused prompt integrated at 2025-10-27 12:30:00)